<a href="https://colab.research.google.com/github/noe2019/Fashion/blob/master/Copy_of_EDA_fashion550k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FASHION550K EXPLANATORY DATA ANALYSIS

This notebook presents an explanatory data analysis (EDA) for Fashion 550k dataset, using fastai library. This library  is implemented in pytorch and has the advantage of being faster than the classical keras and tensorflow. 

The dataset can be dowloaded [here.](https://esslab.jp/~ess//data/Fashion550k_v1.tar.gz).

This dataset consist of 405,589 images downloaded from [Chictopia](http://www.chictopia.com/) with 66 labels.

This EDA was made following the steps bellow:


# 1 Download and Import fastai environnement

In [0]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   665  100   665    0     0   3934      0 --:--:-- --:--:-- --:--:--  3934
    100% |████████████████████████████████| 5.7MB 4.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 22kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6389a000 @  0x7f722e61a2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
Cloning into 'course-v3'...
remote: Enumerating objects: 2809, done.
remote: Total 2809 (delta 0), re

In [0]:
from fastai.vision import*
from PIL import Image
%matplotlib inline

# 2 Grab fashion550k Dataset from website

In [0]:
!wget https://esslab.jp/~ess//data/Fashion550k_v1.tar.gz

--2019-01-11 10:34:47--  https://esslab.jp/~ess//data/Fashion550k_v1.tar.gz
Resolving esslab.jp (esslab.jp)... 133.9.196.180
Connecting to esslab.jp (esslab.jp)|133.9.196.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7738514315 (7.2G) [application/x-gzip]
Saving to: ‘Fashion550k_v1.tar.gz’

Fashion550k_v1.tar. 100%[===================>]   7.21G  28.5MB/s    in 4m 23s  

2019-01-11 10:39:10 (28.1 MB/s) - ‘Fashion550k_v1.tar.gz’ saved [7738514315/7738514315]



#Login to the cloud and upload Fashion 550k on platefom-ai-research

In [0]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&prompt=select_account&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&access_type=offline


Enter verification code: 4/0AAggKI9G8q3cDmUR5-Vz0CZaaqFACo4C8Y-6v-vRXlTZjh9vvNRpjY
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [noe.fouotsa@aims-cameroon.org].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
!gsutil cp ./Fashion550k_v1.tar.gz gs://platform-ai-research/datasets/fashion550k/

Copying file://./Fashion550k_v1.tar.gz [Content-Type=application/x-tar]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/7.2 GiB.                                      


In [0]:
!gsutil -m cp -r gs://platform-ai-research/datasets/fashion550k/Fashion550k_v1.tar.gz .

Copying gs://platform-ai-research/datasets/fashion550k/Fashion550k_v1.tar.gz...
- [1/1 files][  7.2 GiB/  7.2 GiB] 100% Done  21.6 MiB/s ETA 00:00:00           
Operation completed over 1 objects/7.2 GiB.                                      


In [32]:
path = "fashion550k"
os.listdir(path)

FileNotFoundError: ignored

# Untar fashion550k_dataset

In [31]:
import pandas as pd
import cv2
import os
import numpy as np
import matplotlib as plt
import torch
import torchvision
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.vision import *
path = "fashion550k"
os.listdir(path)
files = os.listdir(f'{path}/photos/999')[:5]
files
img = plt.imread(f'{path}/photos/999/{files[3]}')
plt.imshow(img);
img.shape
img[:4,:4]
sz = 224
arch=models.resnet34
data = ImageClassifierData.from_paths(path, tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 2)
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
learn = create_cnn(data, arch, metrics=[acc_02, f_score])

ModuleNotFoundError: ignored

#Creating csv file for proper file_name extension

In [0]:
basedir = "annotation"

In [0]:
# bash command to replace "jpg " by "jpg," to obtain a proper csv
!sed -e 's/jpg /jpg,/g' annotation/labels.txt > annotation/labels.csv

/bin/bash: annotation/labels.csv: No such file or directory


The file **photos.txt** contains the location of images and files names.

We can see the location looking at the number before the slash ""/", it correspond to a path in the photos folder.

As we can see below:

In [0]:
!pip freeze

absl-py==0.6.1
alabaster==0.7.12
albumentations==0.1.9
altair==2.3.0
astor==0.7.1
astropy==3.0.5
atari-py==0.1.7
atomicwrites==1.2.1
attrs==18.2.0
audioread==2.1.6
autograd==1.2
Babel==2.6.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.0.2
bokeh==1.0.3
boto==2.49.0
boto3==1.9.75
botocore==1.12.75
Bottleneck==1.2.1
bs4==0.0.1
bz2file==0.98
cachetools==3.0.0
certifi==2018.11.29
cffi==1.11.5
chainer==5.0.0
chardet==3.0.4
Click==7.0
cloudpickle==0.6.1
cmake==3.12.0
colorlover==0.2.1
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.1.3
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.14.6
cvxopt==1.2.2
cvxpy==1.0.11
cycler==0.10.0
cymem==2.0.2
Cython==0.29.2
cytoolz==0.9.0.1
daft==0.0.4
dask==0.20.2
dataclasses==0.6
datascience==0.10.6
decorator==4.3.0
defusedxml==0.5.0
dill==0.2.8.2
distributed==1.25.2
Django==2.1.5
dlib==19.16.0
dm-sonnet==1.23
docopt==0.6.2
docutils==0.14
dopamine-rl==1.0.5
easydict==1.9
ecos==2.0.7.post1
editdistance==0

In [0]:
!pip freeze